In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.base import TransformerMixin
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.utils import resample
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,roc_auc_score,classification_report,roc_curve,auc, f1_score

import import_ipynb
from function_for_eda import *

C:\Users\Admin\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\Admin\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


importing Jupyter notebook from function_for_eda.ipynb


# 3. PREVIOUS_APPLICATION.CSV
Bảng previous_application chứa các thông tin về các khoản vay trước đó của khách hàng tại Home Credit. Từ các bước EDA trước đó chúng ta tiến hành các bước Feature Engineering như sau:
- <b>Bước 1</b>: Xử lý các điểm bất thường ở các cột chứ DAYS <thay thế bằng giá trị Nan>
- <b>Bước 2</b>: Tạo một số feature mới như Credit-Downpayment Ratio, Amount not approved, Credit to Goods ratio, Interest Rate dựa vào quá trình EDA, domain knowledge và các solutions tham khảo 
- <b>Bước 3 </b>: Thực hiện các aggregations thông qua SK_ID_CURR bằng các phép aggregation như min, max, sum, count,...chúng tôi aggreagated cho toàn bộ data, sau đó chúng tôi aggregate qua 2 bản ghi đầu tiên và 5 bản ghi gần nhất dựa vào cột DAYS_FIRST_DUE.Việc lựa chọn các phép aggregation dựa trên domain knowledge, quá trình eda, cũng như một số solutions tham khảo khác

In [2]:
previous_application = pd.read_csv('../dseb63_final_project_DP_dataset/dseb63_previous_application.csv')
previous_application

,SK_ID_PREV,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,...,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,SK_ID_CURR
0,2030495,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,...,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0,293189
1,1696966,Consumer loans,68258.655,1800000.0,1754721.0,180000.0,1800000.0,SATURDAY,18,Y,...,36.0,low_normal,POS industry with interest,NaN,NaN,NaN,NaN,NaN,NaN,293189
2,2154916,Consumer loans,12417.390,108400.5,119848.5,0.0,108400.5,SUNDAY,14,Y,...,12.0,middle,POS industry with interest,365243.0,-512.0,-182.0,-392.0,-387.0,0.0,293189
3,2802425,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,...,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0,91587
4,1536272,Cash loans,21709.125,450000.0,512370.0,NaN,450000.0,WEDNESDAY,9,Y,...,36.0,low_normal,Cash X-Sell: low,365243.0,-485.0,565.0,-155.0,-147.0,1.0,91587
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1413696,1261221,Consumer loans,6030.675,133866.0,133866.0,0.0,133866.0,SATURDAY,15,Y,...,24.0,low_action,POS household without interest,365243.0,-801.0,-111.0,-531.0,-518.0,0.0,302680
1413697,1563733,Consumer loans,13726.080,137250.0,123525.0,13725.0,137250.0,SATURDAY,14,Y,...,10.0,low_normal,POS household with interest,365243.0,-464.0,-194.0,-194.0,-186.0,0.0,192560
1413698,2319569,Consumer loans,4716.495,46800.0,46800.0,0.0,46800.0,FRIDAY,9,Y,...,12.0,middle,POS industry with interest,365243.0,-119.0,211.0,365243.0,365243.0,0.0,91682
1413699,2829648,Consumer loans,10301.940,101745.0,112491.0,0.0,101745.0,FRIDAY,12,Y,...,12.0,low_action,POS household without interest,365243.0,-529.0,-199.0,-199.0,-194.0,0.0,126653


In [3]:
#sorting the applications from oldest to most recent previous loans for each user
previous_application = previous_application.sort_values(by = ['SK_ID_CURR','DAYS_FIRST_DUE'])

#in the EDA we found some erroneous values in DAYS columns, so we will replace them with NaN values
previous_application['DAYS_FIRST_DRAWING'][previous_application['DAYS_FIRST_DRAWING'] == 365243.0] = np.nan
previous_application['DAYS_FIRST_DUE'][previous_application['DAYS_FIRST_DUE'] == 365243.0] = np.nan
previous_application['DAYS_LAST_DUE_1ST_VERSION'][previous_application['DAYS_LAST_DUE_1ST_VERSION'] == 365243.0] = np.nan
previous_application['DAYS_LAST_DUE'][previous_application['DAYS_LAST_DUE'] == 365243.0] = np.nan
previous_application['DAYS_TERMINATION'][previous_application['DAYS_TERMINATION'] == 365243.0] = np.nan

#we also see abruptly large value for SELLERPLACE_AREA
previous_application['SELLERPLACE_AREA'][previous_application['SELLERPLACE_AREA'] == 4000000] = np.nan

#filling the NaN values
previous_application = fill_nan(previous_application)


In [4]:
name_contract_dict = {'Approved': 0, 'Refused' : 3, 'Canceled' : 2, 'Unused offer' : 1}
previous_application['NAME_CONTRACT_STATUS'] = previous_application['NAME_CONTRACT_STATUS'].map(name_contract_dict)
yield_group_dict = {'XNA': 0, 'low_action': 1, 'low_normal': 2,'middle': 3, 'high': 4}
previous_application['NAME_YIELD_GROUP'] = previous_application['NAME_YIELD_GROUP'].map(yield_group_dict)
appl_per_contract_last_dict = {'Y':1, 'N':0}
previous_application['FLAG_LAST_APPL_PER_CONTRACT'] = previous_application['FLAG_LAST_APPL_PER_CONTRACT'].map(appl_per_contract_last_dict)
remaining_categorical_columns = previous_application.dtypes[previous_application.dtypes == 'object'].index.tolist()
for col in remaining_categorical_columns:
    encoding_dict = dict([(j,i) for i,j in enumerate(previous_application[col].unique(),1)])
    previous_application[col] = previous_application[col].map(encoding_dict)    

    #engineering some features on domain knowledge
    previous_application['MISSING_VALUES_TOTAL_PREV'] = previous_application.isna().sum(axis = 1)
    previous_application['AMT_DECLINED'] = previous_application['AMT_APPLICATION'] - previous_application['AMT_CREDIT']
    previous_application['AMT_CREDIT_GOODS_RATIO'] = previous_application['AMT_CREDIT'] / (previous_application['AMT_GOODS_PRICE'] + 0.00001)
    previous_application['AMT_CREDIT_GOODS_DIFF'] = previous_application['AMT_CREDIT'] - previous_application['AMT_GOODS_PRICE']
    previous_application['AMT_CREDIT_APPLICATION_RATIO'] = previous_application['AMT_APPLICATION'] / (previous_application['AMT_CREDIT'] + 0.00001)
    previous_application['CREDIT_DOWNPAYMENT_RATIO'] = previous_application['AMT_DOWN_PAYMENT'] / (previous_application['AMT_CREDIT'] + 0.00001)
    previous_application['GOOD_DOWNPAYMET_RATIO'] = previous_application['AMT_DOWN_PAYMENT'] / (previous_application['AMT_GOODS_PRICE'] + 0.00001)
    previous_application['INTEREST_DOWNPAYMENT'] = previous_application['RATE_DOWN_PAYMENT'] * previous_application['AMT_DOWN_PAYMENT']
    previous_application['INTEREST_CREDIT'] = previous_application['AMT_CREDIT'] * previous_application['RATE_INTEREST_PRIMARY']
    previous_application['INTEREST_CREDIT_PRIVILEGED'] = previous_application['AMT_CREDIT'] * previous_application['RATE_INTEREST_PRIVILEGED']
    previous_application['APPLICATION_AMT_TO_DECISION_RATIO'] = previous_application['AMT_APPLICATION'] / (previous_application['DAYS_DECISION'] + 0.00001) * -1
    previous_application['AMT_APPLICATION_TO_SELLERPLACE_AREA'] = previous_application['AMT_APPLICATION'] / (previous_application['SELLERPLACE_AREA'] + 0.00001)
    previous_application['ANNUITY'] = previous_application['AMT_CREDIT'] / (previous_application['CNT_PAYMENT'] + 0.00001)
    previous_application['ANNUITY_GOODS'] = previous_application['AMT_GOODS_PRICE'] / (previous_application['CNT_PAYMENT'] + 0.00001)
    previous_application['DAYS_FIRST_LAST_DUE_DIFF' ] = previous_application['DAYS_LAST_DUE'] - previous_application['DAYS_FIRST_DUE']
    previous_application['AMT_CREDIT_HOUR_PROCESS_START'] = previous_application['AMT_CREDIT'] * previous_application['HOUR_APPR_PROCESS_START']
    previous_application['AMT_CREDIT_NFLAG_LAST_APPL_DAY'] = previous_application['AMT_CREDIT'] * previous_application['NFLAG_LAST_APPL_IN_DAY']
    previous_application['AMT_CREDIT_YIELD_GROUP'] = previous_application['AMT_CREDIT'] * previous_application['NAME_YIELD_GROUP']
    
    #https://www.kaggle.com/c/home-credit-default-risk/discussion/64598 <Solution tham khảo>
    previous_application['AMT_INTEREST'] = previous_application['CNT_PAYMENT'] * previous_application[
        'AMT_ANNUITY'] - previous_application['AMT_CREDIT'] 
    previous_application['INTEREST_SHARE'] = previous_application['AMT_INTEREST'] / (previous_application[
        'AMT_CREDIT'] + 0.00001)
    previous_application['INTEREST_RATE'] = 2 * 12 * previous_application['AMT_INTEREST'] / (previous_application[
        'AMT_CREDIT'] * (previous_application['CNT_PAYMENT'] + 1))

In [5]:
aggregations_for_previous_application = {
            'MISSING_VALUES_TOTAL_PREV' : ['sum'],
            'NAME_CONTRACT_TYPE' : ['mean','last'],
            'AMT_ANNUITY' : ['mean','sum','max'],
            'AMT_APPLICATION' : ['mean','max','sum'],
            'AMT_CREDIT' : ['mean','max','sum'],
            'AMT_DOWN_PAYMENT' : ['mean','max','sum'],
            'AMT_GOODS_PRICE' : ['mean','max','sum'],
            'WEEKDAY_APPR_PROCESS_START' : ['mean','max','min'],
            'HOUR_APPR_PROCESS_START' : ['mean','max','min'],
            'FLAG_LAST_APPL_PER_CONTRACT' : ['mean','sum'],
            'NFLAG_LAST_APPL_IN_DAY' : ['mean','sum'],
            'RATE_DOWN_PAYMENT' : ['mean','max'],
            'RATE_INTEREST_PRIMARY' : ['mean','max'],
            'RATE_INTEREST_PRIVILEGED' : ['mean','max'],
            'NAME_CASH_LOAN_PURPOSE' : ['mean','last'],
            'NAME_CONTRACT_STATUS' : ['mean','max','last'],
            'DAYS_DECISION' : ['mean','max','min'],
            'NAME_PAYMENT_TYPE' : ['mean', 'last'],
            'CODE_REJECT_REASON' : ['mean','last'],
            'NAME_TYPE_SUITE' : ['mean','last'],
            'NAME_CLIENT_TYPE' : ['mean','last'],
            'NAME_GOODS_CATEGORY' : ['mean','last'],
            'NAME_PORTFOLIO' : ['mean','last'],
            'NAME_PRODUCT_TYPE' : ['mean','last'],
            'CHANNEL_TYPE' : ['mean','last'],
            'SELLERPLACE_AREA' : ['mean','max','min'],
            'NAME_SELLER_INDUSTRY' : ['mean','last'],
            'CNT_PAYMENT' : ['sum','mean','max'],
            'NAME_YIELD_GROUP' : ['mean','last'],
            'PRODUCT_COMBINATION' : ['mean', 'last'],
            'DAYS_FIRST_DRAWING' : ['mean','max'],
            'DAYS_FIRST_DUE' : ['mean','max'],
            'DAYS_LAST_DUE_1ST_VERSION' : ['mean'],
            'DAYS_LAST_DUE' : ['mean'],
            'DAYS_TERMINATION' : ['mean','max'],
            'NFLAG_INSURED_ON_APPROVAL' : ['sum'],
            'AMT_DECLINED' : ['mean','max','sum'],
            'AMT_CREDIT_GOODS_RATIO' : ['mean', 'max', 'min'],
            'AMT_CREDIT_GOODS_DIFF' : ['sum','mean','max', 'min'],
            'AMT_CREDIT_APPLICATION_RATIO' : ['mean','min'],
            'CREDIT_DOWNPAYMENT_RATIO' : ['mean','max'],
            'GOOD_DOWNPAYMET_RATIO' : ['mean','max'],
            'INTEREST_DOWNPAYMENT' : ['mean','sum','max'],
            'INTEREST_CREDIT' : ['mean','sum','max'],
            'INTEREST_CREDIT_PRIVILEGED' : ['mean','sum','max'],
            'APPLICATION_AMT_TO_DECISION_RATIO' : ['mean','min'],
            'AMT_APPLICATION_TO_SELLERPLACE_AREA' : ['mean','max'],
            'ANNUITY' : ['mean','sum','max'],
            'ANNUITY_GOODS' : ['mean','sum','max'],
            'DAYS_FIRST_LAST_DUE_DIFF' : ['mean','max'],
            'AMT_CREDIT_HOUR_PROCESS_START' : ['mean','sum'],
            'AMT_CREDIT_NFLAG_LAST_APPL_DAY' : ['mean','max'],
            'AMT_CREDIT_YIELD_GROUP' : ['mean','sum','min'],
            'AMT_INTEREST' : ['mean','sum','max','min'],
            'INTEREST_SHARE' : ['mean','max','min'],
            'INTEREST_RATE' : ['mean','max','min']
           }

       

In [6]:
#grouping the previous applications over SK_ID_CURR while only taking the latest 5 applications
group_last_5 = previous_application.groupby('SK_ID_CURR').tail(5).groupby('SK_ID_CURR').agg(aggregations_for_previous_application)
group_last_5.columns = ['_'.join(ele).upper() + '_LAST_5' for ele in group_last_5.columns]

#grouping the previous applications over SK_ID_CURR while only taking the first 2 applications
group_first_2 = previous_application.groupby('SK_ID_CURR').head(2).groupby('SK_ID_CURR').agg(aggregations_for_previous_application)
group_first_2.columns = ['_'.join(ele).upper() + '_FIRST_2' for ele in group_first_2.columns]

#grouping the previous applications over SK_ID_CURR while taking all the applications into consideration
group_all = previous_application.groupby('SK_ID_CURR').agg(aggregations_for_previous_application)
group_all.columns = ['_'.join(ele).upper() + '_ALL' for ele in group_all.columns]

#merging all the applications
previous_application_aggregated = group_last_5.merge(group_first_2, on = 'SK_ID_CURR', how = 'outer')
previous_application_aggregated = previous_application_aggregated.merge(group_all, on = 'SK_ID_CURR', how = 'outer')

In [7]:
previous_application_aggregated.reset_index(inplace = True)

In [8]:
previous_application_aggregated

,SK_ID_CURR,MISSING_VALUES_TOTAL_PREV_SUM_LAST_5,NAME_CONTRACT_TYPE_MEAN_LAST_5,NAME_CONTRACT_TYPE_LAST_LAST_5,AMT_ANNUITY_MEAN_LAST_5,AMT_ANNUITY_SUM_LAST_5,AMT_ANNUITY_MAX_LAST_5,AMT_APPLICATION_MEAN_LAST_5,AMT_APPLICATION_MAX_LAST_5,AMT_APPLICATION_SUM_LAST_5,...,AMT_INTEREST_MEAN_ALL,AMT_INTEREST_SUM_ALL,AMT_INTEREST_MAX_ALL,AMT_INTEREST_MIN_ALL,INTEREST_SHARE_MEAN_ALL,INTEREST_SHARE_MAX_ALL,INTEREST_SHARE_MIN_ALL,INTEREST_RATE_MEAN_ALL,INTEREST_RATE_MAX_ALL,INTEREST_RATE_MIN_ALL
0,0,0,1.200000,1,6699.015000,33495.075000,11180.385000,82333.80000,120658.5,411669.000,...,1244.145000,7.464870e+03,19389.960000,-45000.00,-6.990500e-02,1.690420e-01,-1.000000,-3.808680,0.312077,-24.000000
1,1,0,2.000000,2,13119.747962,65598.739808,15837.184952,9000.00000,45000.0,45000.000,...,219301.083175,1.535108e+06,551591.640000,-45000.00,1.452671e+10,2.542175e+10,-1.000000,inf,inf,-24.000000
2,2,0,1.500000,3,9705.645000,38822.580000,24577.425000,85230.12375,225000.0,340920.495,...,18586.552500,7.434621e+04,55079.100000,3948.66,2.458260e-01,3.640110e-01,0.111000,0.603930,0.955384,0.242181
3,3,0,1.800000,3,9542.905981,47714.529904,15837.184952,43938.00000,135000.0,219690.000,...,65743.806852,3.944628e+05,254217.485556,-135000.00,8.473916e+09,2.542175e+10,-1.000000,inf,inf,-24.000000
4,4,0,1.000000,1,8957.430000,8957.430000,8957.430000,47250.00000,47250.0,47250.000,...,9113.580000,9.113580e+03,9113.580000,9113.58,2.041984e-01,2.041984e-01,0.204198,0.700109,0.700109,0.700109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291052,307505,0,1.250000,1,9132.484988,36529.939952,15837.184952,79915.50000,126891.0,319662.000,...,29070.556389,1.162822e+05,127326.485556,-45000.00,1.378074e-01,1.003432e+00,-1.000000,-5.308273,1.412295,-24.000000
291053,307506,0,1.000000,1,11164.905000,44659.620000,17043.120000,107453.25000,182592.0,429813.000,...,15836.422500,6.334569e+04,28602.720000,2954.25,1.537095e-01,2.093597e-01,0.081200,0.364553,0.700111,0.149908
291054,307508,0,3.000000,3,29052.277981,145261.389904,41087.250000,481500.00000,864000.0,2407500.000,...,116684.320111,1.166843e+06,512496.000000,-900000.00,5.084350e+09,2.542175e+10,-1.000000,inf,inf,-24.000000
291055,307509,0,1.666667,3,9864.630000,29593.890000,16044.075000,121269.00000,229500.0,363807.000,...,22923.360000,6.877008e+04,40663.350000,2434.05,1.881601e-01,2.804011e-01,0.120200,0.378928,0.517663,0.207005


In [9]:
previous_application_aggregated.to_csv('previous_application_final.csv', index = False)
print('done')

done
